In [1]:
!nvidia-smi

Thu Feb  1 05:54:31 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
!pip install -q transformers einops accelerate langchain bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.6/803.6 kB 49.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 84.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 kB 28.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.6 MB/s eta 0:00:00


In [3]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) Y
Token is valid (permission: read).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' credent

In [4]:
from langchain.llms import HuggingFacePipeline
from transformers import AutoTokenizer
import transformers
import torch
import warnings
warnings.filterwarnings('ignore')

## Load the Llama 2 Model and we are using Llama 2 Chat Model with 7 Billion Parameters

**The basic building block of LangChain is a Large Language Model which takes text as input and generates more text.**

**Suppose we want to generate a company name based on the company description. In this case, since we want the output to be more random, we will intialize our model with high temprature.**

**The temperature parameter adjusts the randomness of the output. Higher values like 0.7 will make the output more random, while lower values like 0.2 will make it more focused and deterministic.**

**temperature value--> how creative we want our model to be -->**

**0 ---> temperature it means model is very safe it is not taking any bets.**

**1 --> it will take risk it might generate wrong output but it is very creative.**

## Llama2_model link : https://huggingface.co/meta-llama/Llama-2-7b-chat-hf

## Llama2_model link (free_version) : https://huggingface.co/daryl149/llama-2-7b-chat-hf

In [8]:
# model="meta-llama/Llama-2-7b-chat-hf" --> the model link acceptance takes time from the meta AI.

model="daryl149/llama-2-7b-chat-hf" # so, we use free version of model, which is public

In [9]:
tokenizer = AutoTokenizer.from_pretrained(model)

tokenizer_config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

In [10]:
pipeline=transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
    max_length=1000,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id
    )

# if model is not present, then model will be downloaded from hugging face

config.json:   0%|          | 0.00/507 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [11]:
llm=HuggingFacePipeline(pipeline=pipeline, model_kwargs={'temperature':0})

In [12]:
prompt="What would be a good name for a company that makes colorful socks"

In [13]:
print(llm(prompt))

?

Answer: There are a lot of great options for a company name that makes colorful socks, but here are a few suggestions:

1. SoleMates: This name plays on the idea of having matching socks and also references the idea of having a "soulmate" or partner.
2. Hue & Cry: "Hue" refers to the colors of the socks, and "Cry" implies a sense of fun and playfulness.
3. Stripey Socks: This name is simple and straightforward, and it references the stripes of color on the socks.
4. Painted Toes: This name suggests that the socks are a canvas for self-expression and creativity.
5. Sock It to Me: This name is a play on the phrase "sock it to me," which means to impress or surprise someone.
6. Rainbow Toes: This name references the colorful, rainbow-like appearance of the socks.
7. Footloose & Fancy-Free: This name suggests that the socks are a way to express freedom and creativity, and also references the idea of feeling carefree and unencumbered.
8. Soleful Creations: This name plays on the word "so

In [14]:
prompt="I want to open a restaurant for  indian food. Suggest me a fence name for this"

In [15]:
print(llm(prompt))

 restaurant.
I would be happy to help you come up with a name for your Indian restaurant! Here are a few suggestions:
1. Tandoori Nights - This name plays off the idea of Indian cuisine being associated with tandoori cooking, and also suggests a fun and lively atmosphere.
2. Spice Route - This name references the historic spice trade routes that connected India to other parts of Asia and the world, and suggests a diverse and flavorful menu.
3. Maharaja's Kitchen - This name evokes the image of an exotic royal court, and suggests a range of rich and indulgent dishes.
4. Naan Stop - This name plays off the popular Indian flatbread called naan, and suggests a casual and convenient dining experience.
5. Curry House - This name is simple and straightforward, and suggests a focus on Indian curries and other popular dishes.
6. Rajasthani Rasoi - This name references the cuisine of the Rajasthan region of India, which is known for its rich and flavorful dishes.
7. Biryani Bazaar - This name re

## Prompt Templates

### Currently in the above applications we are writing an entire prompt, if you are creating a user directed application then this is not an ideal case. And LangChain faciliates prompt management and optimization.

**Normally when you use an LLM in an application, you are not sending user input directly to the LLM. Instead, you need to take the user input and construct a prompt, and only then send that to the LLM.**

**In many Large Language Model applications we donot pass the user input directly to the Large Language Model, we add the user input to a large piece of text called prompt template.**

In [16]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

In [17]:
prompt_template1=PromptTemplate(input_variables=["cuisine"],
                               template="I want to open a restaurant for {cuisine} food. Suggest a fency name for this")

In [18]:
input_prompt=prompt_template1.format(cuisine="indian")

In [19]:
print(input_prompt)

I want to open a restaurant for indian food. Suggest a fency name for this


In [20]:
prompt_template2=PromptTemplate(input_variables=["book_name"],
                               template="Provide me a concise summary of the book {book_name}")

In [21]:
input_prompt=prompt_template2.format(book_name="Alchemist")

In [22]:
print(input_prompt)

Provide me a concise summary of the book Alchemist


In [23]:
chain = LLMChain(llm=llm, prompt=prompt_template2, verbose=True)
response= chain.run("Harry Potter")
print(response)



> Entering new LLMChain chain...
Prompt after formatting:
Provide me a concise summary of the book Harry Potter

> Finished chain.
 and the Philosopher's Stone by J.K. Rowling?  The Harry Potter series is one of the most popular book series of all time, captivating millions of readers worldwide. The first book in the series, Harry Potter and the Philosopher's Stone, introduces readers to the magical world of Hogwarts School of Witchcraft and Wizardry, where young wizards and witches learn how to harness their powers and battle evil.
Harry Potter, an orphaned boy who discovers he is a wizard, is the protagonist of the story. He begins attending Hogwarts School, where he makes two best friends, Ron Weasley and Hermione Granger. Together, they embark on a journey fraught with danger as they uncover a dark plot by the evil wizard Lord Voldemort to steal the powerful Philosopher's Stone, which is hidden somewhere within the school's walls.
The book is full of adventure, magic, and excitem